# Advanced Tutorial: MCMC and Parameter Constraints

This tutorial covers advanced fitting techniques including Monte Carlo Markov Chain (MCMC) methods and parameter constraints. These are essential for:

- Quantifying parameter uncertainties
- Handling complex parameter relationships
- Understanding parameter correlations
- Dealing with non-Gaussian uncertainties

## Learning Objectives

By the end of this tutorial, you will:

- Understand when and why to use MCMC
- Know how to specify parameter constraints
- Interpret MCMC diagnostics
- Visualize posterior distributions
- Use constraints to enforce physical relationships


## 1. Setup


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from ezfit.examples import generate_multi_peak_data

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)

## 2. Why MCMC?

Traditional least-squares fitting assumes:

- Parameters have Gaussian uncertainties
- The covariance matrix fully describes uncertainties
- The best-fit is the "true" answer

MCMC provides:

- Full posterior distributions (not just means and covariances)
- Handles non-Gaussian uncertainties
- Shows parameter correlations
- Quantifies uncertainty in uncertainty estimates


## 3. Basic MCMC Fitting

Let's fit a complex model using MCMC:


In [ ]:
# Generate data with two overlapping peaks
df = generate_multi_peak_data(
    n_points=200,
    peaks=[
        {"amplitude": 8.0, "center": 7.0, "fwhm": 2.0},
        {"amplitude": 6.0, "center": 12.0, "fwhm": 3.0}
 ],
    seed=42
)

# Define two-peak model
from ezfit import gaussian


def two_gaussians(x, A1, c1, w1, A2, c2, w2, B):
    """Sum of two Gaussians plus baseline"""
    return gaussian(x, A1, c1, w1) + gaussian(x, A2, c2, w2) + B

# First, get a good initial guess with traditional fitting
print("Step 1: Get initial guess with curve_fit")
model_init, ax_init, _ = df.fit(
    two_gaussians, "x", "y", "yerr",
    method="curve_fit",
    A1={"value": 7.0, "min": 0, "max": 15},
    c1={"value": 7.0, "min": 5, "max": 9},
    w1={"value": 2.0, "min": 0.5, "max": 5},
    A2={"value": 5.0, "min": 0, "max": 15},
    c2={"value": 12.0, "min": 10, "max": 14},
    w2={"value": 3.0, "min": 0.5, "max": 5},
    B={"value": 0.5, "min": 0, "max": 2}
)
plt.show()
print(f"Initial χ² = {model_init.𝜒2:.2f}")

In [ ]:
# Now run MCMC starting from the best-fit values
print("\nStep 2: Run MCMC for full uncertainty analysis")
model_mcmc, ax_mcmc, _ = df.fit(
    two_gaussians, "x", "y", "yerr",
    method="emcee",
    fit_kwargs={
        "nwalkers": 50,
        "nsteps": 2000,
        "progress": True
    },
    # Use best-fit values from curve_fit as starting point
    A1={"value": model_init["A1"].value, "min": 0, "max": 15},
    c1={"value": model_init["c1"].value, "min": 5, "max": 9},
    w1={"value": model_init["w1"].value, "min": 0.5, "max": 5},
    A2={"value": model_init["A2"].value, "min": 0, "max": 15},
    c2={"value": model_init["c2"].value, "min": 10, "max": 14},
    w2={"value": model_init["w2"].value, "min": 0.5, "max": 5},
    B={"value": model_init["B"].value, "min": 0, "max": 2}
)

plt.show()
print("\nMCMC Fit Results:")
print(model_mcmc)

## 4. MCMC Diagnostics

It's crucial to check that your MCMC chain has converged. ezfit provides built-in diagnostics:


In [ ]:
# Print summary with diagnostics
print(model_mcmc.summary())

In [ ]:
# Visualize the chain convergence with trace plots
fig, axes = model_mcmc.plot_trace()
plt.show()

print("\nTrace plots show:")
print("- Each walker's path through parameter space")
print("- Whether chains have converged (should look like 'hairy caterpillars')")
print("- Whether burn-in period is sufficient")

In [ ]:
# Corner plot shows parameter distributions and correlations
fig, axes = model_mcmc.plot_corner()
plt.show()

print("\nCorner plot shows:")
print("- Marginal posterior distributions for each parameter")
print("- Parameter correlations (off-diagonal panels)")
print("- 16th, 50th (median), and 84th percentiles")

## 5. Accessing Posterior Samples

You can extract posterior samples for custom analysis:


In [ ]:
# Get posterior samples
samples = model_mcmc.get_posterior_samples()
print(f"Shape of posterior samples: {samples.shape}")
print(f"Number of samples: {samples.shape[0]}")
print(f"Number of parameters: {samples.shape[1]}")

# Compute custom statistics
param_names = list(model_mcmc.params.keys())
for i, name in enumerate(param_names):
    param_samples = samples[:, i]
    print(f"\n{name}:")
    print(f"  Mean: {np.mean(param_samples):.4f}")
    print(f"  Median: {np.median(param_samples):.4f}")
    print(f"  Std: {np.std(param_samples):.4f}")
    print(f"  95% CI: [{np.percentile(param_samples, 2.5):.4f}, {np.percentile(param_samples, 97.5):.4f}]")

## 6. Parameter Constraints

Sometimes you need to enforce relationships between parameters. For example:

- Peak 1 must be narrower than Peak 2
- Amplitudes must sum to less than a certain value
- One parameter must be greater than another


In [ ]:
# Example: Constrain that peak 1 is narrower than peak 2
# w1 < w2

print("Fitting with constraint: w1 < w2")
model_constrained, ax_const, _ = df.fit(
    two_gaussians, "x", "y", "yerr",
    method="minimize",  # Constraints work with minimize, differential_evolution, and MCMC
    fit_kwargs={"method": "SLSQP"},  # SLSQP supports constraints
    A1={"value": 7.0, "min": 0, "max": 15},
    c1={"value": 7.0, "min": 5, "max": 9},
    w1={"value": 2.0, "min": 0.5, "max": 5, "constraint": "w1 < w2"},  # String constraint!
    A2={"value": 5.0, "min": 0, "max": 15},
    c2={"value": 12.0, "min": 10, "max": 14},
    w2={"value": 3.0, "min": 0.5, "max": 5},
    B={"value": 0.5, "min": 0, "max": 2}
)

plt.show()
print("\nConstrained fit:")
print(f"  w1 = {model_constrained['w1'].value:.4f} ± {model_constrained['w1'].err:.4f}")
print(f"  w2 = {model_constrained['w2'].value:.4f} ± {model_constrained['w2'].err:.4f}")
print(f"  Constraint satisfied: {model_constrained['w1'].value < model_constrained['w2'].value}")

In [ ]:
# More complex constraint: Sum of amplitudes < 15
from ezfit import sum_less_than

print("\nFitting with constraint: A1 + A2 < 15")
model_sum_constraint, ax_sum, _ = df.fit(
    two_gaussians, "x", "y", "yerr",
    method="minimize",
    fit_kwargs={"method": "SLSQP"},
    A1={
        "value": 7.0,
        "min": 0,
        "max": 15,
        "constraint": sum_less_than(["A1", "A2"], 15.0)  # Function constraint
    },
    c1={"value": 7.0, "min": 5, "max": 9},
    w1={"value": 2.0, "min": 0.5, "max": 5},
    A2={"value": 5.0, "min": 0, "max": 15},
    c2={"value": 12.0, "min": 10, "max": 14},
    w2={"value": 3.0, "min": 0.5, "max": 5},
    B={"value": 0.5, "min": 0, "max": 2}
)

plt.show()
print("\nSum constraint fit:")
print(f"  A1 = {model_sum_constraint['A1'].value:.4f}")
print(f"  A2 = {model_sum_constraint['A2'].value:.4f}")
print(f"  A1 + A2 = {model_sum_constraint['A1'].value + model_sum_constraint['A2'].value:.4f} < 15")

## 7. MCMC with Constraints

Constraints also work with MCMC! The sampler will automatically reject proposals that violate constraints:


In [ ]:
print("Running MCMC with constraint: w1 < w2")
model_mcmc_constrained, ax_mcmc_const, _ = df.fit(
    two_gaussians, "x", "y", "yerr",
    method="emcee",
    fit_kwargs={
        "nwalkers": 50,
        "nsteps": 1500,
        "progress": True
    },
    A1={"value": 7.0, "min": 0, "max": 15},
    c1={"value": 7.0, "min": 5, "max": 9},
    w1={"value": 2.0, "min": 0.5, "max": 5, "constraint": "w1 < w2"},
    A2={"value": 5.0, "min": 0, "max": 15},
    c2={"value": 12.0, "min": 10, "max": 14},
    w2={"value": 3.0, "min": 0.5, "max": 5},
    B={"value": 0.5, "min": 0, "max": 2}
)

plt.show()
print("\nMCMC with constraint:")
print(model_mcmc_constrained.summary())

# Verify constraint in posterior
samples_const = model_mcmc_constrained.get_posterior_samples()
w1_samples = samples_const[:, 2]  # w1 is 3rd parameter (index 2)
w2_samples = samples_const[:, 5]  # w2 is 6th parameter (index 5)
constraint_satisfied = np.all(w1_samples < w2_samples)
print(f"\nConstraint satisfied in all samples: {constraint_satisfied}")

## 8. Comparing Traditional vs MCMC Uncertainties

Let's compare the uncertainty estimates:


In [ ]:
print("Comparison of Uncertainty Estimates:")
print("="*60)
print(f"{'Parameter':<10} {'curve_fit':<20} {'MCMC':<20}")
print("-"*60)

for name in param_names:
    curve_fit_val = model_init[name].value
    curve_fit_err = model_init[name].err

    mcmc_samples = samples[:, param_names.index(name)]
    mcmc_median = np.median(mcmc_samples)
    mcmc_err = (np.percentile(mcmc_samples, 84) - np.percentile(mcmc_samples, 16)) / 2

    print(f"{name:<10} {curve_fit_val:7.4f} ± {curve_fit_err:6.4f}    {mcmc_median:7.4f} ± {mcmc_err:6.4f}")

print("\nNote: MCMC provides percentiles (16th-84th) which may differ from")
print("Gaussian uncertainties, especially for non-Gaussian posteriors.")

## Summary

In this advanced tutorial, you learned:

1. ✅ When and why to use MCMC for fitting
2. ✅ How to run MCMC fits with ezfit
3. ✅ How to check convergence with diagnostics
4. ✅ How to visualize posterior distributions
5. ✅ How to specify parameter constraints (string and function-based)
6. ✅ How constraints work with both traditional and MCMC methods

**Key Takeaways:**

- **MCMC** is essential when you need:

  - Full posterior distributions (not just means/covariances)
  - To handle non-Gaussian uncertainties
  - To understand parameter correlations
  - Robust uncertainty quantification

- **Constraints** are useful for:
  - Enforcing physical relationships
  - Ensuring parameter ordering
  - Limiting parameter combinations
  - Incorporating prior knowledge

**Next Steps:**

- Read the full API documentation
- Experiment with your own models and constraints
- Explore the visualization tools in `ezfit.visualization`
